In [2]:
import pandas as pd
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("Kaludi/Customer-Support-Responses")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/74 [00:00<?, ? examples/s]

In [4]:
# Split the train set into train and test sets
train_test_split = dataset['train'].train_test_split(test_size=0.1)  # Adjust the test_size as needed

# Access the new splits
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [5]:
from transformers import T5Tokenizer

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Preprocess function
def preprocess_data(examples):
    inputs = [query for query in examples['query']]
    targets = [response for response in examples['response']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply preprocessing to the datasets
tokenized_train_dataset = train_dataset.map(preprocess_data, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_data, batched=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [6]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,13.189283
2,13.889500,13.011775
3,13.761900,12.730872


TrainOutput(global_step=27, training_loss=13.681808471679688, metrics={'train_runtime': 23.0007, 'train_samples_per_second': 8.608, 'train_steps_per_second': 1.174, 'total_flos': 26797676691456.0, 'train_loss': 13.681808471679688, 'epoch': 3.0})

In [9]:
from datasets import load_metric

# Load metrics
metric = load_metric("rouge")

# Function to generate responses and compute metrics
def evaluate_model():
    predictions = []
    references = []

    for example in eval_dataset:
        inputs = tokenizer(example['query'], return_tensors='pt')
        outputs = model.generate(**inputs)
        predictions.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        references.append(example['response'])

    results = metric.compute(predictions=predictions, references=references)
    return results

# Compute and print evaluation results
results = evaluate_model()
print(results)


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [10]:
import os
import torch
from datasets import load_dataset, load_metric
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the dataset
dataset = load_dataset("Kaludi/Customer-Support-Responses")

# Split the train set into train and test sets
train_test_split = dataset['train'].train_test_split(test_size=0.1)

# Access the new splits
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Preprocess function
def preprocess_data(examples):
    inputs = [query for query in examples['query']]
    targets = [response for response in examples['response']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply preprocessing to the datasets
tokenized_train_dataset = train_dataset.map(preprocess_data, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_data, batched=True)

# Load the model and move it to the GPU if available
model = T5ForConditionalGeneration.from_pretrained('t5-small')
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
metric = load_metric("rouge")

def evaluate_model():
    model.eval()  # Set the model to evaluation mode
    predictions = []
    references = []

    for example in eval_dataset:
        inputs = tokenizer(example['query'], return_tensors='pt')
        inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to the same device as the model
        outputs = model.generate(**inputs)
        predictions.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        references.append(example['response'])

    results = metric.compute(predictions=predictions, references=references)
    return results

# Compute and print evaluation results
results = evaluate_model()
print(results)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,14.447359
2,14.018200,14.269246
3,13.098100,13.973170


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.020833333333333332, recall=0.004464285714285714, fmeasure=0.007352941176470588), mid=Score(precision=0.06944444444444443, recall=0.0182324016563147, fmeasure=0.02879414365215582), high=Score(precision=0.13888888888888887, recall=0.0364648033126294, fmeasure=0.05758828730431165)), 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.015625, recall=0.004310344827586207, fmeasure=0.006756756756756756), high=Score(precision=0.046875, recall=0.01293103448275862, fmeasure=0.020270270270270268)), 'rougeL': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.06944444444444443, recall=0.017857142857142856, fmeasure=0.02879414365215582), high=Score(precision=0.13888888888888887, recall=0.0364648033126294, fmeasure=0.05758828730431165)), 'rougeLsum': AggregateScore(low=Score(precision=0.020833333333333332, recall=0.004464285714285714, fmeasure=0.007352941176470588), m

In [8]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=2f2c4083207bc843997779d2bba00ca90d710ec0708509f37206dd8608c6f530
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [1]:
!pip install datasets